In [1]:
import pandas as pd
import yaml
import itertools
import numpy as np
import math
import json
import pytz
import time
from deep_translator import GoogleTranslator

In [119]:
df = pd.read_csv('China9_5news/death_tweets.csv')
df_o = pd.read_excel('China9_5news/news_api_95_sichuan_content.xlsx')

In [121]:
time_list = df['time']
death_list= df['death_score']
location_list = df['city']
country_list = df['country']
target_country ='China'


In [122]:
id_list = df['origin_id'].tolist()
full_content=df_o['full content'].loc[id_list].tolist()
url_l = df_o['url'].loc[id_list].tolist()

In [123]:
from pandarallel import pandarallel
from math import sin

pandarallel.initialize()

def clean_dict(x):
    tmp_dict = yaml.load(x)
    tmp_list = list(tmp_dict)
    new_dict = {}
    new_dict[tmp_list[0]] = tmp_dict[tmp_list[0]]
    return new_dict

death_list = death_list.parallel_apply(clean_dict)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


In [124]:
def process_loc(df, target_country, location_list,country_list):
    location_binary = []
    geoloc = []
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent='myapplication')
    for k, i in enumerate(location_list):
        try:
            if country_list[k] != target_country and isinstance(country_list[k], str):
                if i and isinstance(i, str):
                    tmp_location = geolocator.geocode(i+" "+country_list[k])
                    if tmp_location and target_country in tmp_location[-2]:
                        location_binary.append(True)
                        geoloc.append(tmp_location[-1])
                    else:
                        location_binary.append(False)
                        geoloc.append(None)
                else:
                    location_binary.append(False)
                    geoloc.append(None)
            elif country_list[k] == target_country:
                if i and isinstance(i, str):
                    tmp_location = geolocator.geocode(i+" "+target_country)
                    location_binary.append(True)
                    tmp_location = [-1] if not tmp_location else tmp_location
#                     print(tmp_location)
                    geoloc.append(tmp_location[-1])
                else:
                    tmp_location = geolocator.geocode(target_country)
                    location_binary.append(True)
                    geoloc.append(tmp_location[-1])
            else:
                if i and isinstance(i, str):
                    tmp_location = geolocator.geocode(i+" "+target_country)
                    location_binary.append(True)
                    geoloc.append(tmp_location[-1])
                else:
                    location_binary.append(False)
                    geoloc.append(None)
        except:
            time.sleep(80) 
            print(k)
    return location_binary, geoloc

In [125]:
def process_data(df, time_list, death_list, location_binary):
    num_list = []
    prob_list = []
    first_num_list = []
    first_prob_list = []
    for i, tmp_time in enumerate(time_list):
        if location_binary[i]:
            tmp_dict = death_list[i]
            nan_key_list = []
            for tmp_key_0 in tmp_dict.keys():
                if tmp_key_0 == 'none':
                    tmp_key_0 = '-1'
                if not tmp_key_0.isnumeric() and not tmp_key_0 == '-1':
                    nan_key_list.append(tmp_key_0)
            for tmp_key_0 in nan_key_list:
                del tmp_dict[tmp_key_0]
            tmp_len = len(tmp_dict.keys())
            tmp_list_num = []
            tmp_list_prob = []
            if tmp_len <= 1:
                for tmp_token, tmp_dict_2 in tmp_dict.items():
                    if tmp_token == 'none':
                        tmp_token = '-1'
                    if tmp_token.isnumeric() or tmp_token == '-1':
                        for tmp_token_2, tmp_prob in tmp_dict_2.items():
                            if tmp_token_2 == 'none':
                                tmp_token_2 = '-1'
                            if tmp_token_2.isnumeric() or tmp_token_2 == '-1':
                                tmp_list_num.append(int(tmp_token_2))
                                tmp_list_prob.append(tmp_prob) 

            else:
                count_up = min(2, 5 - tmp_len)
                tmp_list_n = []
                tmp_list_p = []
                for tmp_token, tmp_dict_2 in tmp_dict.items():
                    tmp_list_num_2 = []
                    tmp_list_prob_2 = []
                    if tmp_token == 'none':
                        tmp_token = '-1'
                    if tmp_token.isnumeric() or tmp_token == '-1':
                        count = 0
                        for tmp_token_2, tmp_prob in tmp_dict_2.items():
                            if count < count_up:
                                if tmp_token_2 == 'none' or tmp_token_2 == '-1':
                                    tmp_token_2 = '-1'
                                if tmp_token_2.isnumeric():
                                    tmp_list_num_2.append(tmp_token_2)

                                    tmp_list_prob_2.append(tmp_prob) 
                                    count += 1

                    tmp_list_n.append(tmp_list_num_2)
                    tmp_list_p.append(tmp_list_prob_2)

                count = 0
                tmp_list_n = list(itertools.product(*tmp_list_n))
                tmp_list_p = list(itertools.product(*tmp_list_p))
                for i, item_n in enumerate(tmp_list_n):
                    tmp_str = "".join(n for n in item_n if n )
                    tmp_list_num.append(int(tmp_str) if tmp_str else -1)
                    tmp_list_prob.append(np.prod(tmp_list_p[i]))

            new_tmp_list_p = [x for x,y in sorted(zip(tmp_list_prob,tmp_list_num),key=lambda x: x[0],reverse=True)]

            new_tmp_list_n = [y for x,y in sorted(zip(tmp_list_prob, tmp_list_num),key=lambda x: x[0],reverse=True)]
            
            tmp_l = min(len(new_tmp_list_n), 5)
            num_list.append(new_tmp_list_n[:tmp_l])
            prob_list.append(new_tmp_list_p[:tmp_l])
            if len(new_tmp_list_n) > 0:
                first_num_list.append(new_tmp_list_n[0])
            else:
                first_num_list.append(None)
            if len(new_tmp_list_p) > 0:
                first_prob_list.append(new_tmp_list_p[0])
            else:
                first_prob_list.append(None)
        else:
                num_list.append(None)
                prob_list.append(None)
                first_num_list.append(None)
                first_prob_list.append(None)
    return num_list, prob_list, first_num_list, first_prob_list
    

In [126]:
geo_results = process_loc(df, target_country, location_list,country_list)
location_binary, geoloc = geo_results
location_binary = [True]*len(location_binary)

In [128]:
# the wiki request sometimes will break when reaching the upper limit, I setup resume after 80 seconds, but you will need to drop the missed rows to match.
# drop_list = [70]
# drop_list = [5213,5927]#731
# drop_list = [473,970,1153]#729
# drop_list = [2806,10267]#728
# new_time_list = time_list.drop(time_list.index[drop_list])
# new_death_list = death_list.drop(death_list.index[drop_list])
# new_time_list.reset_index(drop=True, inplace=True)
# new_death_list.reset_index(drop=True, inplace=True)

In [129]:
new_time_list = time_list
new_death_list = death_list

In [130]:
results = process_data(df, new_time_list, new_death_list,location_binary)

In [131]:
num_list, prob_list, first_num_list, first_prob_list = results
num_s = pd.Series(num_list)
prob_s = pd.Series(prob_list)
first_num_s = pd.Series(first_num_list)
first_prob_s = pd.Series(first_prob_list)
geo_s = pd.Series(geoloc)
loc_b_s = pd.Series(location_binary)
df['variables_death'] = num_s
df['probs_death'] = prob_s
df['variables_top1_death'] = first_num_s
df['probs_top1_death'] = first_prob_s
df['geo'] = geo_s
df['location_correct?'] = loc_b_s
df['deaths'] = first_num_s

In [132]:
death_list= df['injury_score']
death_list = death_list.parallel_apply(clean_dict)
# new_death_list = death_list.drop(death_list.index[drop_list])
# new_death_list.reset_index(drop=True, inplace=True)
new_death_list = death_list
results_2 = process_data(df, new_time_list, new_death_list,location_binary)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


In [133]:
num_list, prob_list, first_num_list, first_prob_list = results_2
num_s = pd.Series(num_list)
prob_s = pd.Series(prob_list)
first_num_s = pd.Series(first_num_list)
first_prob_s = pd.Series(first_prob_list)
geo_s = pd.Series(geoloc)
loc_b_s = pd.Series(location_binary)
df['variables_injury'] = num_s
df['probs_injury'] = prob_s
df['variables_top1_injury'] = first_num_s
df['probs_top1_injury'] = first_prob_s
df['injurues'] = first_num_s

In [134]:
new_time = pd.to_datetime(df['time'],format="%Y-%m-%dT%H:%M:%SZ", errors='coerce', utc=False)
utc_time = new_time.dt.tz_localize('US/Eastern').dt.tz_convert('UTC')
df['UTC_time'] = utc_time.dt.tz_localize(None)
df['full_content'] = pd.Series(full_content)
df['url'] = pd.Series(url_l)


In [135]:
df.to_csv('China9_5news/death_tweets_processed.csv')
pd.DataFrame.to_excel(df,'China9_5news/death_tweets_processed.xlsx')